# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способ заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from razdel import tokenize
import math
import numpy as np

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [4]:
def razdel_tokenizer(text):
    lst = list(tokenize(text))
    return [word.text for word in lst]

In [1009]:
default_vect = TfidfVectorizer(min_df=10, max_df=0.4)

In [1010]:
X_default = default_vect.fit_transform(train.comment)
X_default_test = default_vect.transform(test.comment)

In [1011]:
razdel_vect = TfidfVectorizer(min_df=10, max_df=0.4, tokenizer = razdel_tokenizer)

In [1012]:
X_razdel = razdel_vect.fit_transform(train.comment)
X_razdel_test = razdel_vect.transform(test.comment)

In [1013]:
X_razdel.shape, X_razdel_test.shape

((12970, 3448), (1442, 3448))

In [1014]:
X_default.shape, X_default_test.shape

((12970, 3383), (1442, 3383))

In [1015]:
y = train.toxic.values
y_test = test.toxic.values

In [1017]:
classifier = DecisionTreeClassifier(max_depth=8, class_weight='balanced')

In [1018]:
classifier.fit(X_default, y)
preds_default = classifier.predict(X_default_test)
print(classification_report(y_test, preds_default))

              precision    recall  f1-score   support

         0.0       0.82      0.50      0.62       982
         1.0       0.42      0.76      0.54       460

    accuracy                           0.58      1442
   macro avg       0.62      0.63      0.58      1442
weighted avg       0.69      0.58      0.59      1442



In [1019]:
classifier.fit(X_razdel, y)
preds_razdel = classifier.predict(X_razdel_test)
print(classification_report(y_test, preds_razdel))

              precision    recall  f1-score   support

         0.0       0.84      0.41      0.55       982
         1.0       0.40      0.83      0.54       460

    accuracy                           0.54      1442
   macro avg       0.62      0.62      0.54      1442
weighted avg       0.70      0.54      0.54      1442



Вывод: классификатор с токенизацией razdel в целом чуть лучше с точки зрения f1 и recall, чем классификатор со встроенной токенизацией, precision почти не отличается. Хотя прогоны каждый раз дают разные результаты, так что можно сказать, что разницы вовсе нет или она очень несущественна.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [1020]:
df = pd.DataFrame({"я" : [1, 1, 3, 1, 0],
                 "ты" : [1, 1, 0, 0, 0],
                 "и" : [1, 1, 1, 0, 0],
                 "только" : [0, 0, 1, 1, 0],
                 "не" : [0, 0, 0, 1, 0],
                 "он" : [0, 0, 0, 0, 1]})
df = df.rename({0 : "я и ты", 1 : "ты и я", 2 : "я, я и только я", 3 : "только не я", 4 : "он"})
df

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [1021]:
def calculate_tfidf(df):
    values = []
    for row in df.index:
        doc_tfidf = []
        for column in df.columns:
            # берем значение в ячейке:
            doc_tfidf.append(df[column].loc[row] / \
            # делим на количество слов в тексте (сумма по строке) - получаем tf
            df.loc[row].sum() * \
            # tf умножаем на idf: логарифм от частного - количество текстов в коллекции (первая размерность матрицы)
            math.log(df.shape[0] / \
                     # делим на количество текстов в коллекции с данным токеном (количество ненулевых значений в столбце)
                     np.count_nonzero(df[column])))
        values.append(doc_tfidf)
    return pd.DataFrame(values, index = df.index, columns = df.columns)

In [1022]:
calculate_tfidf(df)

,я,ты,и,только,не,он
я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [10]:
count = CountVectorizer(max_df=0.35, max_features=3000, binary=True, tokenizer=razdel_tokenizer, ngram_range=(1,2))
tfidf = TfidfVectorizer(min_df=10, max_df=0.4, sublinear_tf=True, smooth_idf=False, ngram_range=(1,2))

In [11]:
train_2, test_2 = train_test_split(data, test_size=0.1, shuffle=True)

In [12]:
X_count_train = count.fit_transform(train_2.comment)
X_count_test = count.transform(test_2.comment)

In [13]:
X_count_train.shape, X_count_test.shape

((12970, 3000), (1442, 3000))

In [14]:
X_tfidf_train = tfidf.fit_transform(train_2.comment)
X_tfidf_test = tfidf.transform(test_2.comment)

In [15]:
X_tfidf_train.shape, X_tfidf_test.shape

((12970, 4645), (1442, 4645))

In [18]:
y_train = train_2.toxic.values
y_test = test_2.toxic.values

In [17]:
lr = LogisticRegression(C=0.2, class_weight='balanced')

In [1119]:
lr.fit(X_count_train, y_train)
preds_count = lr.predict(X_count_test)
print(classification_report(y_test, preds_count))

              precision    recall  f1-score   support

         0.0       0.90      0.78      0.83       930
         1.0       0.68      0.84      0.75       512

    accuracy                           0.80      1442
   macro avg       0.79      0.81      0.79      1442
weighted avg       0.82      0.80      0.80      1442



In [1120]:
nb = MultinomialNB(alpha=0.7, fit_prior = False)

In [1121]:
nb.fit(X_tfidf_train, y_train)
preds_tfidf = nb.predict(X_tfidf_test)
print(classification_report(y_test, preds_tfidf))

              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88       930
         1.0       0.78      0.78      0.78       512

    accuracy                           0.84      1442
   macro avg       0.83      0.83      0.83      1442
weighted avg       0.84      0.84      0.84      1442



In [1122]:
probas_toxic = pd.DataFrame()
probas_toxic['comment'] = test_2.comment
probas_toxic['toxic_probas'] = nb.predict_proba(X_tfidf_test)[:,1]
for i in range(10):
    print(probas_toxic.sort_values('toxic_probas', ascending = False).iloc[i][0], 'Вероятность:', 
         probas_toxic.sort_values('toxic_probas', ascending = False).iloc[i][1], "| Метка:",
          test_2.toxic.values[test_2.comment == probas_toxic.sort_values('toxic_probas', ascending = False).iloc[i][0]][0], 
          end = '\n\n')
# все комментарии действительно токсичны

Какие же хохлы всадники, пиздец просто.
 Вероятность: 0.9978602747497146 | Метка: 1.0

По мексикански Флаг: Ублюдок, мать твою, а ну иди сюда говно собачье, решил меня поднять? Ты, засранец вонючий, мать твою, а? Ну иди сюда, попробуй меня поднять, я тебя сам подниму ублюдок, онанист чертов, будь ты проклят, иди идиот, трахать тебя и всю семью, говно собачье, жлоб вонючий, дерьмо, сука, падла, иди сюда, мерзавец, негодяй, гад, иди сюда ты - говно, ЖОПА!
 Вероятность: 0.9970673815700962 | Метка: 1.0

Какие же хохлы новозеландцы дегенераты.
 Вероятность: 0.99644339264494 | Метка: 1.0

null 0 Сука, какие же коммибляди тупые.
 Вероятность: 0.9951049763907913 | Метка: 1.0

Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!
 Вероятность: 0.9942895248242882 | Метка: 1.0

ВОЖДЬ Я УБЬЮ ТЕБЯ СУКА
 Вероятность: 0.9902176083514301 | Метка: 1.0

Мерзкий, тупой но сука смешной
 Вероятность: 0.9887629671893426 | Метка: 1.0

АХАХАХАХАХАХАХ Ты в очередной раз доказываешь нас

In [1124]:
probas_toxic_1 = pd.DataFrame()
probas_toxic_1['comment'] = test_2.comment
probas_toxic_1['toxic_probas'] = lr.predict_proba(X_count_test)[:,1]
for i in range(10):
    print(probas_toxic_1.sort_values('toxic_probas', ascending = False).iloc[i][0], 'Вероятность:', 
         probas_toxic_1.sort_values('toxic_probas', ascending = False).iloc[i][1], '| метка:',
          test_2.toxic.values[test_2.comment == probas_toxic_1.sort_values('toxic_probas', ascending = False).iloc[i][0]][0], 
          end = '\n\n')
# один большой комментарий (первый) на самом деле не является токсичным (по метке) 
# возможно, он был отнесен в эту категорию из-за большого количества мата
# остальные действительно токсичны
# только одно совпадение: "Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!"
# почему-то в топе у байеса в основном короткие комментарии, у лог. регрессии - длинные и средней длины

та ну, хуйня это все про джентельменство . меня в свое время эти брачные игрища так сильно заебали, что я вообще перестал париться и начал себя вести так, как и обычно. без всех этих ужимок и образа славного парня . без зазрения совести перед девушками курил, харкался, матерился как сапожник, рассказывал охуитительные истории о том, как когда-то обосрался бухал ебал кого-то проснулся с обблеванной тян пересказывал пасты про говно и т.д. т.е вел себя как быдлан, как обычно я себя и веду, когда вокруг никого нет. и тогда девушки начали тянуться. после этого я сделал для себя вывод, что девушки вообще не воспринимают что ты им говоришь, что делаешь и т.д. они воспринимают лишь то, как ты все это говоришь и делаешь. когда я начал вести себя как обычно, я делал это, блять, уверенно, самовлюбленно, нагло и аки самец . может это будило в девушках какие-то там инстинкты и они думали что-то в стиле ну ничего что быдло, зато какой уверенный, ВАХ просто! или я хуй знает, но такое поведение позвол

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [21]:
train_3, test_3 = train_test_split(data, test_size=0.1, shuffle=True)
stop = stopwords.words('russian')
vect = CountVectorizer(max_df=0.35, max_features=3000, binary=True, tokenizer=razdel_tokenizer, stop_words=stop)

In [22]:
train_X = vect.fit_transform(train_3.comment)
test_X = vect.transform(test_3.comment)

In [23]:
train_y = train_3.toxic.values
test_y = test_3.toxic.values

In [235]:
nbc = MultinomialNB(alpha=0.7)
nbc.fit(train_X, train_y)
preds_nb = nbc.predict(test_X)

In [238]:
# создаем словарь, где ключ - индекс, значение - важность признака
# по индексу сопоставляем каждому слову показатель его важности
nbc_values = dict(enumerate(nbc.feature_log_prob_[0]))
dict_nbc = {}
for w in vect.vocabulary_:
    dict_nbc[w] = nbc_values[vect.vocabulary_[w]]
# отсортируем словарь по значению
toxic_nbc = {}
sorted_keys = sorted(dict_nbc, key=dict_nbc.get) 
for w in sorted_keys:
    toxic_nbc[w] = dict_nbc[w]

list(toxic_nbc.items())[:5]

[('даун', -11.718928670541132),
 ('ебаные', -11.718928670541132),
 ('пидарасы', -11.718928670541132),
 ('сука', -11.718928670541132),
 ('пидорашка', -11.718928670541132)]

In [239]:
lreg = LogisticRegression(C=0.2, class_weight='balanced')
lreg.fit(train_X, train_y)
preds_lr = lreg.predict(test_X)

In [240]:
lreg_values = dict(enumerate(lreg.coef_[0]))
dict_lreg = {}
for w in vect.vocabulary_:
    dict_lreg[w] = lreg_values[vect.vocabulary_[w]]

toxic_lreg = {}
sorted_keys = sorted(dict_lreg, key=dict_lreg.get, reverse=True) 
for w in sorted_keys:
    toxic_lreg[w] = dict_lreg[w]

list(toxic_lreg.items())[:5]

[('хохлы', 1.890095370868237),
 ('хохлов', 1.8751993397986741),
 ('быдло', 1.4229512603162613),
 ('русских', 1.4127919975985022),
 ('тупые', 1.3900318761908554)]

In [241]:
dt = DecisionTreeClassifier()
dt.fit(train_X, train_y)
preds_dt = dt.predict(test_X)

In [242]:
dt_values = dict(enumerate(dt.feature_importances_))
dict_dt = {}
for w in vect.vocabulary_:
    dict_dt[w] = dt_values[vect.vocabulary_[w]]

toxic_dt = {}
sorted_keys_dt = sorted(dict_dt, key=dict_dt.get, reverse=True) 
for w in sorted_keys_dt:
    toxic_dt[w] = dict_dt[w]

list(toxic_dt.items())[:5]

[(')', 0.027452048986981083),
 ('хохлы', 0.01437896049204086),
 ('тебе', 0.01301802053921191),
 ('нахуй', 0.012753267425720673),
 ('хохлов', 0.01152831771037857)]

In [243]:
rf = RandomForestClassifier()
rf.fit(train_X, train_y)
preds_rf = rf.predict(test_X)

In [244]:
rf_values = dict(enumerate(rf.feature_importances_))
dict_rf = {}
for w in vect.vocabulary_:
    dict_rf[w] = rf_values[vect.vocabulary_[w]]

toxic_rf = {}
sorted_keys_rf = sorted(dict_rf, key=dict_rf.get, reverse=True) 
for w in sorted_keys_rf:
    toxic_rf[w] = dict_rf[w]

list(toxic_rf.items())[:5]

[(')', 0.012651777926934919),
 ('хохлы', 0.01106267011651788),
 ('хохлов', 0.009616974301070353),
 ('тебе', 0.008097603706890636),
 ('(', 0.00792177600095888)]